In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 1. 데이터 로딩
# 전동킥보드 운영 현황: 자치구별 업체별 총 대수, 운영 자치구 정보
devices = pd.read_csv('서울시 민간대여 공유 전동킥보드 기기 현황_25.2월기준.csv')
# 견인 데이터: 자치구별 견인 건수
tow = pd.read_csv('서울특별시_전동킥보드_견인_현황_20220731 (1).csv')

# 2. 자치구별 운영 기기 수 추정 (업체별 총 대수 / 운영 자치구 개수, 균등분배)
gu_device = {}
for idx, row in devices.iterrows():
    districts = str(row['운영지역']).replace(' ', '').split(',')
    n = len(districts)
    each = int(row['기기대수(대)'] // n)
    for gu in districts:
        gu_device[gu] = gu_device.get(gu, 0) + each

gu_device = pd.Series(gu_device).reset_index()
gu_device.columns = ['자치구', '기초공급대수']

# 3. 견인 가중치 반영 (최대 1,000대까지 가산)
tow_group = tow.groupby('자치구')['견인건수'].sum().reset_index()
tow_group['견인가중치'] = tow_group['견인건수'] / tow_group['견인건수'].max() * 1000

# 4. 자치구별 총 배치량(기초공급대수 + 견인가중치)
gu_info = pd.merge(gu_device, tow_group[['자치구', '견인가중치']], on='자치구', how='left')
gu_info['총배치량'] = gu_info['기초공급대수'] + gu_info['견인가중치'].fillna(0)

# 5. 날짜, 시간별 데이터 생성
dates = pd.date_range('2021-01-01', '2025-12-31', freq='D')
hours = list(range(24))

# 6. 가중치 함수 정의
def get_season_weight(month):
    # 계절별 가중치
    if month in [12, 1, 2]:
        return 0.8
    elif month in [5, 6, 9, 10]:
        return 1.2
    else:
        return 1.0

def get_weekend_weight(weekday):
    # 주말 가중치
    return 1.3 if weekday >= 5 else 1.0

def get_hour_weight(hour):
    # 시간대 가중치
    if 7 <= hour <= 9 or 17 <= hour <= 19:
        return 1.3  # 출퇴근 시간
    elif 0 <= hour <= 5:
        return 0.6  # 심야
    elif 10 <= hour <= 16:
        return 1.0  # 주간
    elif 20 <= hour <= 23:
        return 0.9  # 저녁
    else:
        return 1.0

# 7. 시간별 PM 배치량 산출
result = []
for date in dates:
    month = date.month
    weekday = date.weekday()  # 0=월, 6=일
    season_w = get_season_weight(month)
    weekend_w = get_weekend_weight(weekday)
    for gu_idx, gu_row in gu_info.iterrows():
        # 일별 총 배치량 (자치구별), 계절/요일 가중치 적용
        daily_total = gu_row['총배치량'] * season_w * weekend_w
        for hour in hours:
            hour_w = get_hour_weight(hour)
            # 시간별 배치량 (1/24 분배 × 시간 가중치 적용)
            hour_batch = daily_total / 24 * hour_w
            result.append({
                '일시': datetime(date.year, date.month, date.day, hour),
                '자치구': gu_row['자치구'],
                '시간별_PM_배치수': int(np.round(hour_batch))
            })

df_result = pd.DataFrame(result)

# 8. 결과 저장
df_result.to_csv('서울시_시간별_PM_배치_2021_2025_생성.csv', index=False)

# ----------- 주석 정리 -------------
# - 기초공급대수: 업체별 기기수를 운영자치구 수로 균등 분배하여 자치구별 추정
# - 견인가중치: 자치구별 견인 건수 비율로 0~1,000대 가중치 산출, 공급수요 높음 반영
# - 계절/요일/시간대별 가중치: 시계열적으로 PM 수요의 변동 반영
# - 하루 24시간, 자치구별, 날짜별, 시간별로 배치수 생성
# - 결과 컬럼: [일시, 자치구, 시간별_PM_배치수]

# 가중치 데이터 확인:
print("계절가중치: 12~2월=0.8, 5,6,9,10월=1.2, 기타=1.0")
print("주말가중치: 주말=1.3, 평일=1.0")
print("시간대가중치: 출퇴근(7~9, 17~19)=1.3, 심야(0~5)=0.6, 주간(10~16)=1.0, 저녁(20~23)=0.9")
print("견인가중치(최대):", gu_info['견인가중치'].max())
print(df_result.head())
